In [52]:
import os
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import dominate
from selenium.webdriver.remote.webelement import WebElement as Element
del os.environ['DISPLAY']


KeyError: 'DISPLAY'

In [2]:
from pathlib import Path

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_argument("--remote-debugging-port=9222")
driver = webdriver.Chrome(options=options)

In [171]:
driver.get("http://ec2-3-131-244-37.us-east-2.compute.amazonaws.com:7770/")
driver.implicitly_wait(0) # seconds

In [59]:
JS_BUILD_CSS_SELECTOR = ("for(var e=arguments[0],n=[],i=function(e,n){if(!e||!n)return 0;f"
    "or(var i=0,a=e.length;a>i;i++)if(-1==n.indexOf(e[i]))return 0;re"
    "turn 1};e&&1==e.nodeType&&'HTML'!=e.nodeName;e=e.parentNode){if("
    "e.id){n.unshift('#'+e.id);break}for(var a=1,r=1,o=e.localName,l="
    "e.className&&e.className.trim().split(/[\\s,]+/g),t=e.previousSi"
    "bling;t;t=t.previousSibling)10!=t.nodeType&&t.nodeName==e.nodeNa"
    "me&&(i(l,t.className)&&(l=null),r=0,++a);for(var t=e.nextSibling"
    ";t;t=t.nextSibling)t.nodeName==e.nodeName&&(i(l,t.className)&&(l"
    "=null),r=0);n.unshift(r?o:o+(l?'.'+l.join('.'):':nth-child('+a+'"
    ")'))}return n.join(' > ');")



In [270]:
import dominate.tags


clickables = {}

def set_attribute(element: Element, attribute, value):
    driver.execute_script("arguments[0].setAttribute(arguments[1], arguments[2]);", element, attribute, value)

def register_clickable(element: Element, name: str):
    clickables[name] = element
    set_attribute(element, 'data-clickable-id', name)

def get_text(element):
    elementText = element.text # sometime NOT work
    if not elementText:
        elementText = element.get_attribute("innerText")
    if not elementText:
        elementText = element.get_attribute("textContent")
    return element.get_attribute("innerText")


actions = {}

def process(element: Element, recipe, parent_name = ''):
    elementText = ''
    if 'text_selector' in recipe:
        text_element = element.find_element(By.CSS_SELECTOR, recipe['text_selector'])
        elementText = get_text(text_element)
    else:
        elementText = get_text(element)
    if 'add_text' in recipe and recipe['add_text']:
        if not elementText:
            elementText = get_text(element)
    if 'text_format' in recipe and recipe['text_format']:
        elementText = recipe['text_format'].format(elementText)
    
    tag_name = element.tag_name
    if 'tag_name' in recipe:
        tag_name = recipe['tag_name']
    node = getattr(dominate.tags, tag_name)(elementText if 'add_text' in recipe and recipe['add_text'] else '')
    
    if 'name' in recipe and recipe['name']:
        if recipe['name'] == 'from_text':
            if elementText:
                element_name = elementText.lower().replace(' ', '_')
                for special_char in "[]{}()<>.:;|!@#$%^&*+-=,?/\\\"'":
                    element_name = element_name.replace(special_char, '')
                node['name'] = (parent_name + "." if parent_name else '') + element_name
                parent_name = node['name']
            else:
                raise Exception('name from_text must have add_text')
        else:
            node['name'] = (parent_name + "." if parent_name else '') + recipe['name']
        parent_name = node['name']
    if 'clickable' in recipe and recipe['clickable']:
        if 'name' not in recipe:
            raise Exception('clickable element must have a name')
        register_clickable(element, node['name'])
    for key in ['alt', 'src', 'href', 'title', 'type', 'value']:
        value = element.get_dom_attribute(key)
        if value:
            node[key] = value
    for key in ['class', 'id']:
        if key in recipe and recipe[key]:
            node[key] = recipe[key]
    if 'radio' in recipe and recipe['radio']:
        if element.get_attribute('checked'):
            node.text += ' (selected)'
    if 'children' in recipe and recipe['children']:
        with node:
            for child in recipe['children']:
                if 'direct_child' in child and child['direct_child']:
                    selector = ':scope > ' + child['selector']
                else:
                    selector = child['selector']
                elements = element.find_elements(By.CSS_SELECTOR, selector)
                for child_element in elements:
                    process(child_element, child, parent_name)
    return node


In [273]:
recipe = {
    'selector': 'html',
    'clickable': False,
    'children': [
        {
            'selector': 'head',
            'name': '',
            'children': [
                {
                    'selector': 'title',
                    'add_text': True,
                }
            ]
        },
        {
            'selector': 'body',
            'children': [
                # {
                #     'selector': 'nav',
                #     'name': 'nav',
                #     'children': [
                #         {
                #             'selector': 'ul',
                #             'action': 'strip_add_children',
                #             'direct_child': True,
                #             'children': [
                #                 {
                #                     'selector': 'li',
                #                     'direct_child': True,
                #                     'add_text': True,
                #                     'text_selector': 'a',
                #                     'clickable': True,
                #                     'children': [
                #                         {
                #                             'selector': 'ul',
                #                             'direct_child': True,
                #                             'children': [
                #                                 {
                #                                     'selector': 'li',
                #                                     'add_text': True,
                #                                     'direct_child': True,
                #                                     'text_selector': 'a',
                #                                     'clickable': True,
                #                                     'children': [
                #                                         {
                #                                             'selector': 'ul',
                #                                             'direct_child': True,
                #                                             'children': [
                #                                                 {
                #                                                     'selector': 'li',
                #                                                     'add_text': True,
                #                                                     'direct_child': True,
                #                                                     'text_selector': 'a',
                #                                                     'clickable': True,
                #                                                 }
                #                                             ]
                #                                         }
                #                                     ]
                #                                 }
                #                             ]
                #                         }
                #                     ]
                #                 }
                #             ]
                #         }
                #     ]
                # },
                {
                    'selector': '#maincontent > div.columns > div > div:nth-child(3)',
                    'add_text': True,
                    'text_selector': 'div > div.block-title > strong',
                    'children': [
                        {
                            'selector': 'div.product-item-info',
                            'class': 'product-item-info',
                            'name': 'from_text',
                            'text_selector': 'div.product-item-details strong.product-item-name',
                            'children': [
                                {
                                    'selector': 'img',
                                },
                                {
                                    'selector': 'div.product-item-details',
                                    'children': [
                                        {
                                            'selector': 'div.rating-summary > div > span > span',
                                            'add_text': True,
                                            'name': 'rating',
                                            'clickable': True,
                                            'text_format': 'Rating: {}',
                                            'name': 'view_reviews'
                                        },
                                        {
                                            'selector': '.product-item-name',
                                            'add_text': True,
                                            'clickable': True,
                                            'name': 'view_product'
                                        },
                                        {
                                            'selector': '.price-box',
                                            'add_text': True,
                                        },
                                        {
                                            'selector': '.actions-primary',
                                            'add_text': True,
                                            'clickable': True,
                                            'name':  'add_to_card',
                                            'tag_name': 'button'
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}

In [274]:
print(process(driver.find_element(By.CSS_SELECTOR, recipe['selector']), recipe))

<html>
  <head>
    <title>One Stop Market</title>
  </head>
  <body>
    <div>Product Showcases
      <div class="product-item-info" name="prebaked_gingerbread_house_kit_value_pack_17_oz_pack_of_2_total_34_oz">
        <img alt="Image" src="http://ec2-3-131-244-37.us-east-2.compute.amazonaws.com:7770/media/catalog/product/cache/89ff578b9cd87e0600daac45c9e1ea98/B/0/B08PCSHBXY.0.jpg">
        <div>
          <span name="prebaked_gingerbread_house_kit_value_pack_17_oz_pack_of_2_total_34_oz.view_reviews">Rating: 20%</span>
          <strong name="prebaked_gingerbread_house_kit_value_pack_17_oz_pack_of_2_total_34_oz.view_product">Pre-baked Gingerbread House Kit Value Pack, 17 oz., Pack of 2, Total 34 oz.</strong>
          <div>$19.99</div>
          <button name="prebaked_gingerbread_house_kit_value_pack_17_oz_pack_of_2_total_34_oz.add_to_card">Add to Cart</button>
        </div>
      </div>
      <div class="product-item-info" name="v8_energy_healthy_energy_drink_steady_energy_from_blac

In [ ]:
print(len(driver.find_element(By.CSS_SELECTOR, 'html').find_element(By.CSS_SELECTOR, 'nav').find_element(By.CSS_SELECTOR, 'ul').find_elements(By.CSS_SELECTOR, 'li')))

301


In [198]:
print(clickables)

{'beauty_&_personal_care': <selenium.webdriver.remote.webelement.WebElement (session="fb40a98df1e22d9246fdf35d537a2abf", element="f.61335BC334089FFDCEA69597859FA8DD.d.704552B28F5BCF7ADC1B205D741C19FD.e.622")>, 'sports_&_outdoors': <selenium.webdriver.remote.webelement.WebElement (session="fb40a98df1e22d9246fdf35d537a2abf", element="f.61335BC334089FFDCEA69597859FA8DD.d.704552B28F5BCF7ADC1B205D741C19FD.e.687")>, 'clothing,_shoes_&_jewelry': <selenium.webdriver.remote.webelement.WebElement (session="fb40a98df1e22d9246fdf35d537a2abf", element="f.61335BC334089FFDCEA69597859FA8DD.d.704552B28F5BCF7ADC1B205D741C19FD.e.697")>, 'home_&_kitchen': <selenium.webdriver.remote.webelement.WebElement (session="fb40a98df1e22d9246fdf35d537a2abf", element="f.61335BC334089FFDCEA69597859FA8DD.d.704552B28F5BCF7ADC1B205D741C19FD.e.711")>, 'office_products': <selenium.webdriver.remote.webelement.WebElement (session="fb40a98df1e22d9246fdf35d537a2abf", element="f.61335BC334089FFDCEA69597859FA8DD.d.704552B28F5BCF

In [136]:
title = driver.find_element(By.CSS_SELECTOR, 'html').find_element(By.CSS_SELECTOR, 'head').find_element(By.CSS_SELECTOR, 'title')

In [63]:
title.get_attribute('innerHTML')

'One Stop Market'

In [64]:
type(title)

selenium.webdriver.remote.webelement.WebElement

In [65]:
driver.execute_script(JS_BUILD_CSS_SELECTOR, title)

'head > title'

In [77]:
btn = driver.find_element(By.CSS_SELECTOR, '#maincontent > div.columns > div > div:nth-child(3) > div > div.block-content > div.products-grid.grid > ol > li:nth-child(3) > div > div > div.product-item-inner > div > div.actions-primary > form > button')


In [79]:
btn.setProperty('data-clickable-id', '123')

AttributeError: 'WebElement' object has no attribute 'setProperty'

In [83]:
btn.get_attribute('data-clickable-id')

'123'

In [92]:
btn.get_attribute('src')

In [98]:
title = dominate.tags.title('title')

In [114]:
print(title)

<title checked="True">title (selected)
  <div>test</div>
</title>


In [111]:
title.text += ' (selected)'

In [113]:
with title:
    dominate.tags.div('test')